In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
mainpath = "/content/"
filename = "bankmod.csv"
fullpath = os.path.join(mainpath, filename)

In [ ]:
data = pd.read_csv(fullpath,sep=",")
data

,age,job,marital,month,dayofweek,previous,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,target
0,56,housemaid,married,may,mon,0,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,may,mon,0,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,may,mon,0,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,may,mon,0,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,may,mon,0,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10648,73,retired,married,nov,fri,0,-1.1,94.767,-50.8,1.028,4963.6,1
10649,46,blue-collar,married,nov,fri,0,-1.1,94.767,-50.8,1.028,4963.6,0
10650,56,retired,married,nov,fri,0,-1.1,94.767,-50.8,1.028,4963.6,0
10651,44,technician,married,nov,fri,0,-1.1,94.767,-50.8,1.028,4963.6,1


###Estandarización de las variables númericas

In [ ]:
from sklearn.preprocessing import StandardScaler
data[['age', 'previous','empvarrate','conspriceidx',"consconfidx","euribor3m","nremployed"]]=StandardScaler().fit_transform(data[['age', 'previous','empvarrate','conspriceidx',"consconfidx","euribor3m","nremployed"]])

### Conversión de las variables categóricas a númericas(categorización)

In [ ]:
data.job=data.job.astype("category").cat.codes
data.marital=data.marital.astype("category").cat.codes
data.month=data.month.astype("category").cat.codes
data.dayofweek=data.dayofweek.astype("category").cat.codes

In [ ]:
data.head()

,age,job,marital,month,dayofweek,previous,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,target
0,1.323661,3,1,6,1,-0.48371,1.221893,1.415527,0.934807,1.107212,0.610008,0
1,1.411120,7,1,6,1,-0.48371,1.221893,1.415527,0.934807,1.107212,0.610008,0
2,-0.338054,7,1,6,1,-0.48371,1.221893,1.415527,0.934807,1.107212,0.610008,0
3,-0.075678,0,1,6,1,-0.48371,1.221893,1.415527,0.934807,1.107212,0.610008,0
4,1.323661,7,1,6,1,-0.48371,1.221893,1.415527,0.934807,1.107212,0.610008,0


In [ ]:
bank_data_vars = data.columns.values.tolist()
Y = ['target']
predictors = [v for v in bank_data_vars if v not in Y]

In [ ]:
X = data[predictors]
Y = data["target"]

## 2. Comienzo la red neuronal

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:
import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#Creación de la red neuronal
model = Sequential()
# Primera capa 
model.add(Dense(11, input_dim=11, activation='relu'))
# Capa oculta
model.add(Dense(5, activation='relu'))
# Capa de salida
model.add(Dense(1, activation='sigmoid'))

# Compilo el modelo
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size = 11, epochs = 30)

Epoch 1/30
678/678 [==============================] - 1s 1ms/step - loss: 0.6464 - accuracy: 0.6119
Epoch 2/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3770 - accuracy: 0.8462
Epoch 3/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3382 - accuracy: 0.8673
Epoch 4/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3176 - accuracy: 0.8749
Epoch 5/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3147 - accuracy: 0.8757
Epoch 6/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3182 - accuracy: 0.8713
Epoch 7/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3264 - accuracy: 0.8667
Epoch 8/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3133 - accuracy: 0.8769
Epoch 9/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3165 - accuracy: 0.8725
Epoch 10/30
678/678 [==============================] - 1s 1ms/step - loss: 0.3038 - accuracy: 0.8717

In [ ]:
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.50)

## Matriz de Confusión y AUC

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics
cm = confusion_matrix(Y_test, Y_pred)
print("\nMatriz de confusión: \n{}".format(cm))
 
print("\nAccuracy score: \n{}".format(metrics.accuracy_score(Y_test, Y_pred)))


Matriz de confusión: 
[[1721   91]
 [ 324 1060]]

Accuracy score: 
0.8701501877346683


In [ ]:
TN=cm[0][0]
TP=cm[1][1]
FN=cm[0][1]
FP=cm[1][0]

In [ ]:
sens = TP/(TP+FN)
sens

0.9209383145091226

In [ ]:
espc_1 = TN/(TN+FP)
espc_1

0.8415647921760391

In [ ]:
from sklearn.metrics import roc_auc_score
print("AUC on Test data is " +str(roc_auc_score(Y_test,Y_pred)))

AUC on Test data is 0.8578376016026746


## Resumen de los predictores de clasificación

In [ ]:
from sklearn.metrics import classification_report
print("=== Classification Report ===")
print(classification_report(Y_test, Y_pred))
print('\n')

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      1812
           1       0.92      0.77      0.84      1384

    accuracy                           0.87      3196
   macro avg       0.88      0.86      0.86      3196
weighted avg       0.88      0.87      0.87      3196



